# Summarizing legal documents with Hugging Face and Amazon Sagemaker

In [ ]:
# Number of parameters for flan-t5 family: small 80M, base 250M, large 780M, xl 3B, xxl 11B
model_id = "google/flan-t5-large"

# https://huggingface.co/datasets/billsum
dataset_id = "billsum"

# Setup

In [ ]:
!pip -q install transformers datasets sagemaker --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 97.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.4 MB/s eta 0:00:00


In [ ]:
!pip -q install widgetsnbextension ipywidgets

In [ ]:
class sagemaker.model.Model(model_data, image, role, predictor_cls=None, env=None, name=None, sagemaker_session=None)


In [ ]:
import sagemaker

print(sagemaker.__version__)

sess = sagemaker.Session()
bucket = sess.default_bucket()

In [ ]:
import transformers
import datasets

print(transformers.__version__)
print(datasets.__version__)

4.37.2
2.17.1


# Preprocessing

## Load dataset

In [ ]:
from datasets import load_dataset, load_from_disk

dataset = load_dataset(dataset_id)
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 18949
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 3269
    })
    ca_test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 1237
    })
})

## Preprocess dataset

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)

prefix = "summarize: "
input_max_length = 1024
output_max_length = 128


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=input_max_length, truncation=True)
    labels = tokenizer(
        text_target=examples["summary"], max_length=output_max_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
tokenized_dataset = dataset.map(
    preprocess_function, batched=True, remove_columns=["title", "text", "summary"]
)

Map:   0%|          | 0/18949 [00:00<?, ? examples/s]

Map:   0%|          | 0/3269 [00:00<?, ? examples/s]

Map:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset.save_to_disk(f"billsum-t5-tokenized")

Saving the dataset (0/1 shards):   0%|          | 0/18949 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3269 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1237 [00:00<?, ? examples/s]

# Upload processed dataset to S3

In [ ]:
pip install datasets


In [ ]:
!pip install fileSystem

In [ ]:
from datasets.filesystems import S3FileSystem


In [ ]:
from datasets.filesystems import S3FileSystem

s3 = S3FileSystem()

s3_prefix = "huggingface/billsum-t5-summarization"

dataset_input_path = "s3://{}/{}".format(bucket, s3_prefix)
train_input_path = "{}/train".format(dataset_input_path)
valid_input_path = "{}/validation".format(dataset_input_path)

print(dataset_input_path)
print(train_input_path)
print(valid_input_path)

In [ ]:
tokenized_dataset["train"].save_to_disk(train_input_path, fs=s3)
tokenized_dataset["test"].save_to_disk(valid_input_path, fs=s3)

In [ ]:
#%%sh -s $dataset_input_path
#aws s3 ls --recursive $1

In [ ]:
#train_ds = load_from_disk(train_input_path)
#valid_ds = load_from_disk(valid_input_path)

# Fine-tune on SageMaker with a Hugging Face Deep Learning Container

In [ ]:
!pygmentize train.py

In [ ]:
hyperparameters = {
    "epochs": 1,
    "learning-rate": 1e-6,
    "train-batch-size": 1,
    "eval-batch-size": 8,
    "model-name": model_id,
}

In [ ]:
from sagemaker.huggingface import HuggingFace

huggingface_estimator = HuggingFace(
    role=sagemaker.get_execution_role(),
    # Fine-tuning script
    entry_point="train.py",
    dependencies=["requirements.txt"],
    hyperparameters=hyperparameters,
    # Infrastructure
    transformers_version="4.28.1",
    pytorch_version="2.0.0",
    py_version="py310",
    instance_type="ml.g5.xlarge",
    instance_count=1,
    #use_spot_instances=True,
    #max_run=86400, # 24 hours
    #max_wait=86400,
)

In [ ]:
huggingface_estimator.fit({"train": train_input_path, "valid": valid_input_path})

In [ ]:
huggingface_estimator.model_data

# Copy model and predict

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM
from datasets import load_dataset

model_id = "juliensimon/flan-t5-large-billsum-qlora"
base_model_id = "google/flan-t5-large"

config = PeftConfig.from_pretrained(model_id)
base_model = AutoModelForSeq2SeqLM.from_pretrained(base_model_id)
model = PeftModel.from_pretrained(base_model, model_id)
model.eval()

In [ ]:
dataset = load_dataset("billsum")
sample = dataset['test'][123]

In [ ]:
sample['text']

In [ ]:
input_ids = tokenizer(sample["text"], return_tensors="pt", truncation=True).input_ids

outputs = model.generate(input_ids=input_ids, max_new_tokens=64, do_sample=True, top_p=0.8)

tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]